In [157]:
import matplotlib
matplotlib.use("Agg")
#  import the necessary packages
from keras.layers.core import Dropout
from keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.models import Sequential
from keras.layers.core import Dense
from keras.optimizers import SGD
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
import pickle
import cv2
from PIL import Image
import os

In [158]:
# initialize the data ,labels and datagenrator
datagen = ImageDataGenerator()
data = []
labels = []

In [159]:
# grab the image paths and randomly shuffle them
imagePaths = sorted(list(paths.list_images('images/')))
random.seed(42)
random.shuffle(imagePaths)

## Argumentation, Resize, Flatten and append to data,label lists
### Flattening: 64x64x3 = 12288

In [160]:
for imagePath in imagePaths:
	image = cv2.imread(imagePath)
	imageo = image
	datagen.apply_transform(imageo,{'theta':5,'shear':5,'zx':0.1,'zy':0.1})
	datagen.apply_transform(imageo,{'flip_horizontal':True})
	datagen.apply_transform(imageo,{'zoom_rage':0.2})
	image1 = cv2.resize(imageo, (64, 64)).flatten()    
	data.append(image1)
	label1 = imagePath.split(os.path.sep)[-2]
	labels.append(label1)
# flatten the image into 64x64x3 = 12288
	image2 = cv2.resize(image, (64, 64)).flatten()
	data.append(image2)
	label2 = imagePath.split(os.path.sep)[-2]
# extract the class label from the image path and update the labels list
	labels.append(label2)

In [161]:
data[:1][0]

array([255, 255, 255, ...,  49,  83, 137], dtype=uint8)

In [162]:
len(data),len(labels)

(226, 226)

## Scale the raw pixel intensities to the range [0, 1] from [0, 255]

In [163]:
# scale the raw pixel intensities to the range [0, 1]
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

## Train, Test split of 75, 25

In [164]:
# partition the data into training and testing splits using 75% of
# the data for training and the remaining 25% for testing
(trainX, testX, trainY, testY) = train_test_split(data,
	labels, test_size=0.25, random_state=42)

In [165]:
# convert the labels from integers to vectors (for 2-class, binary
# classification you should use Keras' to_categorical function
# instead as the scikit-learn's LabelBinarizer will not return a
# vector)
lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
testY = lb.transform(testY)

In [166]:
# define the 3072-1024-128-3 architecture using Keras
model = Sequential()
model.add(Dense(1024, input_shape=(12288,), activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(128, activation="relu")) #Hidden 1
model.add(Dropout(0.2))
model.add(Dense(len(lb.classes_), activation="softmax")) #How many predictions do you want?

In [167]:
# initialize our initial learning rate and # of epochs to train for
INIT_LR = 0.007
EPOCHS = 40
# compile the model using SGD as our optimizer and categorical
# cross-entropy loss (you'll want to use binary_crossentropy
# for 2-class classification)
print("[INFO] training network...")
opt = SGD(lr=INIT_LR)
model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])

[INFO] training network...


In [168]:
# train the neural network
H = model.fit(trainX, trainY, validation_data=(testX, testY),
	epochs=EPOCHS, batch_size=32)

Train on 169 samples, validate on 57 samples
Epoch 1/40
169/169 [==============================] - 1s 9ms/step - loss: 1.7449 - acc: 0.3373 - val_loss: 1.9842 - val_acc: 0.3333
Epoch 2/40
169/169 [==============================] - 1s 5ms/step - loss: 1.5511 - acc: 0.3787 - val_loss: 1.6478 - val_acc: 0.2807
Epoch 3/40
169/169 [==============================] - 1s 5ms/step - loss: 1.4775 - acc: 0.3669 - val_loss: 2.1346 - val_acc: 0.1579
Epoch 4/40
169/169 [==============================] - 1s 3ms/step - loss: 1.4756 - acc: 0.4497 - val_loss: 1.5155 - val_acc: 0.3158
Epoch 5/40
169/169 [==============================] - 1s 4ms/step - loss: 1.3257 - acc: 0.4911 - val_loss: 1.4909 - val_acc: 0.2982
Epoch 6/40
169/169 [==============================] - 1s 3ms/step - loss: 1.3793 - acc: 0.3787 - val_loss: 1.3990 - val_acc: 0.4386
Epoch 7/40
169/169 [==============================] - 1s 4ms/step - loss: 1.2081 - acc: 0.5148 - val_loss: 1.2909 - val_acc: 0.3860
Epoch 8/40
169/169 [===========

In [169]:
# evaluate the network
print("[INFO] evaluating network...")
predictions = model.predict(testX, batch_size=32)
print(classification_report(testY.argmax(axis=1),
	predictions.argmax(axis=1), target_names=lb.classes_))
# plot the training loss and accuracy
N = np.arange(0, EPOCHS)
plt.style.use("ggplot")
plt.figure()
plt.plot(N, H.history["loss"], label="train_loss")
plt.plot(N, H.history["val_loss"], label="val_loss")
plt.plot(N, H.history["acc"], label="train_acc")
plt.plot(N, H.history["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy (Simple NN)")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend()
plt.savefig('stats.png')

[INFO] evaluating network...
                     precision    recall  f1-score   support

       akshay_kumar       0.80      1.00      0.89         8
       katrina_kaif       0.73      1.00      0.85        11
nawazuddin_siddiqui       1.00      0.83      0.91        12
      ranbir_kapoor       0.83      0.83      0.83        12
        zarine_khan       1.00      0.71      0.83        14

          micro avg       0.86      0.86      0.86        57
          macro avg       0.87      0.88      0.86        57
       weighted avg       0.89      0.86      0.86        57

